Downloading IMDB Dataset

In [43]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz
!rm -r aclImdb/train/unsup

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  39.0M      0  0:00:02  0:00:02 --:--:-- 39.0M


**Preparing the data**

In [44]:
shutil.rmtree('aclImdb/val')

In [45]:
import os, pathlib, shutil, random
from tensorflow import keras
batch_size = 32
base_dir = pathlib.Path("aclImdb")
val_dir = base_dir / "val"
train_n = base_dir / "train_n"
train_dir = base_dir / "train"
for category in ("neg", "pos"):
    os.makedirs(val_dir / category)
    files = os.listdir(train_dir / category)
    random.Random(1337).shuffle(files)
    num_val_samples = 10000
    val_files = files[-num_val_samples:]
    for fname in val_files:
        shutil.move(train_dir / category / fname,
                    val_dir / category / fname)

In [46]:
shutil.rmtree('aclImdb/train_n')

# Training Sample Size: 100

In [3]:
for category in ("neg", "pos"):
    os.makedirs(train_n / category)
    files = os.listdir(train_dir / category)
    num_train_samples=100
    train_files = files[-num_train_samples:]
    for fname in train_files:
        shutil.move(train_dir / category / fname,
                    train_n / category / fname)

train_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train_n", batch_size=batch_size
)
val_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/val", batch_size=batch_size
)
test_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/test", batch_size=batch_size
)
text_only_train_ds = train_ds.map(lambda x, y: x)

Found 200 files belonging to 2 classes.
Found 20000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


**Preparing integer sequence datasets**

In [4]:
from tensorflow.keras import layers

max_length = 150
max_tokens = 10000
text_vectorization = layers.TextVectorization(
    max_tokens=max_tokens,
    output_mode="int",
    output_sequence_length=max_length,
)
text_vectorization.adapt(text_only_train_ds)

int_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
int_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
int_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

**A sequence model built on one-hot encoded vector sequences**

In [5]:
import tensorflow as tf
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = tf.one_hot(inputs, depth=max_tokens)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="relu")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="adam",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 tf.one_hot (TFOpLambda)     (None, None, 10000)       0         
                                                                 
 bidirectional (Bidirection  (None, 64)                2568448   
 al)                                                             
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 2568513 (9.80 MB)
Trainable params: 2568513 (9.80 MB)
Non-trainable params: 0 (0.00 Byte)
_______________________

**Training a first basic sequence model**

In [6]:
callbacks = [
    keras.callbacks.ModelCheckpoint("one_hot_bidir_lstm.keras",
                                    save_best_only=True,
                                    monitor="val_loss")
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)
model = keras.models.load_model("one_hot_bidir_lstm.keras")
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Epoch 1/10
7/7 [==============================] - 15s 1s/step - loss: 2.1738 - accuracy: 0.5000 - val_loss: 1.1229 - val_accuracy: 0.5000
Epoch 2/10
7/7 [==============================] - 8s 1s/step - loss: 0.9642 - accuracy: 0.5000 - val_loss: 0.7997 - val_accuracy: 0.5000
Epoch 3/10
7/7 [==============================] - 8s 1s/step - loss: 0.6849 - accuracy: 0.5900 - val_loss: 0.7405 - val_accuracy: 0.5009
Epoch 4/10
7/7 [==============================] - 8s 1s/step - loss: 0.6825 - accuracy: 0.6900 - val_loss: 0.6886 - val_accuracy: 0.5305
Epoch 5/10
7/7 [==============================] - 8s 1s/step - loss: 0.5683 - accuracy: 0.7200 - val_loss: 0.6932 - val_accuracy: 0.5419
Epoch 6/10
7/7 [==============================] - 8s 1s/step - loss: 0.5318 - accuracy: 0.7700 - val_loss: 0.6829 - val_accuracy: 0.5523
Epoch 7/10
7/7 [==============================] - 8s 1s/step - loss: 0.4688 - accuracy: 0.8600 - val_loss: 0.7164 - val_accuracy: 0.5120
Epoch 8/10
7/7 [========================

**Instantiating an `Embedding` layer**

In [7]:
embedding_layer = layers.Embedding(input_dim=max_tokens, output_dim=256)

**Model that uses an `Embedding` layer trained from scratch**

In [9]:
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = layers.Embedding(input_dim=max_tokens, output_dim=256)(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="relu")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="adam",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()



Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_2 (Embedding)     (None, None, 256)         2560000   
                                                                 
 bidirectional_2 (Bidirecti  (None, 64)                73984     
 onal)                                                           
                                                                 
 dropout_2 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 2634049 (10.05 MB)
Trainable params: 2634049 (10.05 MB)
Non-trainable params: 0 (0.00 Byte)
___________________

In [10]:
callbacks = [
    keras.callbacks.ModelCheckpoint("embeddings_bidir_gru.keras",
                                    save_best_only=True,
                                    monitor="val_loss")
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)
model = keras.models.load_model("embeddings_bidir_gru.keras")
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Epoch 1/10
7/7 [==============================] - 9s 897ms/step - loss: 2.7652 - accuracy: 0.5000 - val_loss: 1.0563 - val_accuracy: 0.5000
Epoch 2/10
7/7 [==============================] - 5s 778ms/step - loss: 0.8373 - accuracy: 0.5050 - val_loss: 0.8095 - val_accuracy: 0.5121
Epoch 3/10
7/7 [==============================] - 5s 778ms/step - loss: 0.6271 - accuracy: 0.6000 - val_loss: 0.7543 - val_accuracy: 0.5098
Epoch 4/10
7/7 [==============================] - 5s 781ms/step - loss: 0.5455 - accuracy: 0.7250 - val_loss: 0.7173 - val_accuracy: 0.5220
Epoch 5/10
7/7 [==============================] - 5s 778ms/step - loss: 0.4401 - accuracy: 0.8400 - val_loss: 0.6961 - val_accuracy: 0.5424
Epoch 6/10
7/7 [==============================] - 5s 761ms/step - loss: 0.3323 - accuracy: 0.9150 - val_loss: 0.8220 - val_accuracy: 0.5616
Epoch 7/10
7/7 [==============================] - 4s 720ms/step - loss: 0.1128 - accuracy: 0.9850 - val_loss: 0.7040 - val_accuracy: 0.5792
Epoch 8/10
7/7 [====

#### Understanding padding and masking

**Using an `Embedding` layer with masking enabled**

In [11]:
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = layers.Embedding(
    input_dim=max_tokens, output_dim=256, mask_zero=True)(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="relu")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="adam",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()



Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_3 (Embedding)     (None, None, 256)         2560000   
                                                                 
 bidirectional_3 (Bidirecti  (None, 64)                73984     
 onal)                                                           
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 2634049 (10.05 MB)
Trainable params: 2634049 (10.05 MB)
Non-trainable params: 0 (0.00 Byte)
___________________

In [12]:
callbacks = [
    keras.callbacks.ModelCheckpoint("embeddings_bidir_gru_with_masking.keras",
                                    save_best_only=True)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)
model = keras.models.load_model("embeddings_bidir_gru_with_masking.keras")
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Epoch 1/10
7/7 [==============================] - 15s 1s/step - loss: 2.3211 - accuracy: 0.5000 - val_loss: 1.1389 - val_accuracy: 0.5000
Epoch 2/10
7/7 [==============================] - 5s 842ms/step - loss: 0.9436 - accuracy: 0.5000 - val_loss: 0.8777 - val_accuracy: 0.5000
Epoch 3/10
7/7 [==============================] - 5s 849ms/step - loss: 0.6729 - accuracy: 0.5200 - val_loss: 0.7610 - val_accuracy: 0.5000
Epoch 4/10
7/7 [==============================] - 5s 841ms/step - loss: 0.5258 - accuracy: 0.6900 - val_loss: 0.6961 - val_accuracy: 0.5257
Epoch 5/10
7/7 [==============================] - 5s 820ms/step - loss: 0.4187 - accuracy: 0.8550 - val_loss: 0.7101 - val_accuracy: 0.5333
Epoch 6/10
7/7 [==============================] - 5s 800ms/step - loss: 0.2904 - accuracy: 0.9200 - val_loss: 0.7043 - val_accuracy: 0.5480
Epoch 7/10
7/7 [==============================] - 5s 831ms/step - loss: 0.2006 - accuracy: 0.9550 - val_loss: 0.6754 - val_accuracy: 0.5795
Epoch 8/10
7/7 [======

#### Using pretrained word embeddings

In [13]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip

--2024-05-05 03:11:57--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2024-05-05 03:11:57--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2024-05-05 03:11:57--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

**Parsing the GloVe word-embeddings file**

In [14]:
import numpy as np
path_to_glove_file = "glove.6B.100d.txt"

embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print(f"Found {len(embeddings_index)} word vectors.")

Found 400000 word vectors.


**Preparing the GloVe word-embeddings matrix**

In [15]:
embedding_dim = 100

vocabulary = text_vectorization.get_vocabulary()
word_index = dict(zip(vocabulary, range(len(vocabulary))))

embedding_matrix = np.zeros((max_tokens, embedding_dim))
for word, i in word_index.items():
    if i < max_tokens:
        embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [ ]:
embedding_layer = layers.Embedding(
    max_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
    mask_zero=True,
)

**Model that uses a pretrained Embedding layer**

In [16]:
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = embedding_layer(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="relu")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="adam",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

callbacks = [
    keras.callbacks.ModelCheckpoint("glove_embeddings_sequence_model.keras",
                                    save_best_only=True,
                                    monitor="val_loss")
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)
model = keras.models.load_model("glove_embeddings_sequence_model.keras")
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 256)         2560000   
                                                                 
 bidirectional_4 (Bidirecti  (None, 64)                73984     
 onal)                                                           
                                                                 
 dropout_4 (Dropout)         (None, 64)                0         
                                                                 
 dense_4 (Dense)             (None, 1)                 65        
                                                                 
Total params: 2634049 (10.05 MB)
Trainable params: 2634049 (10.05 MB)
Non-trainable params: 0 (0.00 Byte)
___________________

# Training Sample Size: 500

In [21]:
for category in ("neg", "pos"):
    os.makedirs(train_n / category)
    files = os.listdir(train_dir / category)
    num_train_samples=500
    train_files = files[-num_train_samples:]
    for fname in train_files:
        shutil.move(train_dir / category / fname,
                    train_n / category / fname)

train_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train_n", batch_size=batch_size
)
val_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/val", batch_size=batch_size
)
test_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/test", batch_size=batch_size
)
text_only_train_ds = train_ds.map(lambda x, y: x)

Found 1000 files belonging to 2 classes.
Found 20000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [22]:
from tensorflow.keras import layers

max_length = 150
max_tokens = 10000
text_vectorization = layers.TextVectorization(
    max_tokens=max_tokens,
    output_mode="int",
    output_sequence_length=max_length,
)
text_vectorization.adapt(text_only_train_ds)

int_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
int_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
int_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

**A sequence model built on one-hot encoded vector sequences**

In [23]:
import tensorflow as tf
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = tf.one_hot(inputs, depth=max_tokens)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="relu")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="adam",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, None)]            0         
                                                                 
 tf.one_hot_1 (TFOpLambda)   (None, None, 10000)       0         
                                                                 
 bidirectional_5 (Bidirecti  (None, 64)                2568448   
 onal)                                                           
                                                                 
 dropout_5 (Dropout)         (None, 64)                0         
                                                                 
 dense_5 (Dense)             (None, 1)                 65        
                                                                 
Total params: 2568513 (9.80 MB)
Trainable params: 2568513 (9.80 MB)
Non-trainable params: 0 (0.00 Byte)
_____________________

**Training a first basic sequence model**

In [24]:
callbacks = [
    keras.callbacks.ModelCheckpoint("one_hot_bidir_lstm.keras",
                                    save_best_only=True,
                                    monitor="val_loss")
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)
model = keras.models.load_model("one_hot_bidir_lstm.keras")
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Epoch 1/10
32/32 [==============================] - 13s 303ms/step - loss: 1.1661 - accuracy: 0.4940 - val_loss: 0.7151 - val_accuracy: 0.5011
Epoch 2/10
32/32 [==============================] - 9s 281ms/step - loss: 0.6633 - accuracy: 0.5960 - val_loss: 0.6619 - val_accuracy: 0.6384
Epoch 3/10
32/32 [==============================] - 9s 282ms/step - loss: 0.5503 - accuracy: 0.7950 - val_loss: 0.6216 - val_accuracy: 0.6769
Epoch 4/10
32/32 [==============================] - 9s 280ms/step - loss: 0.4602 - accuracy: 0.7960 - val_loss: 0.6057 - val_accuracy: 0.6971
Epoch 5/10
32/32 [==============================] - 9s 278ms/step - loss: 0.2135 - accuracy: 0.9630 - val_loss: 0.6627 - val_accuracy: 0.6472
Epoch 6/10
32/32 [==============================] - 9s 277ms/step - loss: 0.1836 - accuracy: 0.9570 - val_loss: 1.8036 - val_accuracy: 0.7305
Epoch 7/10
32/32 [==============================] - 9s 276ms/step - loss: 0.0321 - accuracy: 0.9990 - val_loss: 1.9854 - val_accuracy: 0.7341
Epoch

**Model that uses an `Embedding` layer trained from scratch**

In [25]:
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = layers.Embedding(input_dim=max_tokens, output_dim=256)(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="relu")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="adam",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()



Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_4 (Embedding)     (None, None, 256)         2560000   
                                                                 
 bidirectional_6 (Bidirecti  (None, 64)                73984     
 onal)                                                           
                                                                 
 dropout_6 (Dropout)         (None, 64)                0         
                                                                 
 dense_6 (Dense)             (None, 1)                 65        
                                                                 
Total params: 2634049 (10.05 MB)
Trainable params: 2634049 (10.05 MB)
Non-trainable params: 0 (0.00 Byte)
___________________

In [26]:
callbacks = [
    keras.callbacks.ModelCheckpoint("embeddings_bidir_gru.keras",
                                    save_best_only=True,
                                    monitor="val_loss")
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)
model = keras.models.load_model("embeddings_bidir_gru.keras")
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Epoch 1/10
32/32 [==============================] - 11s 252ms/step - loss: 1.0445 - accuracy: 0.5060 - val_loss: 0.6846 - val_accuracy: 0.5515
Epoch 2/10
32/32 [==============================] - 7s 237ms/step - loss: 0.5915 - accuracy: 0.7050 - val_loss: 0.6936 - val_accuracy: 0.5792
Epoch 3/10
32/32 [==============================] - 6s 205ms/step - loss: 0.3520 - accuracy: 0.9030 - val_loss: 1.4653 - val_accuracy: 0.6497
Epoch 4/10
32/32 [==============================] - 6s 194ms/step - loss: 0.1582 - accuracy: 0.9640 - val_loss: 1.0432 - val_accuracy: 0.6002
Epoch 5/10
32/32 [==============================] - 6s 188ms/step - loss: 0.0637 - accuracy: 0.9780 - val_loss: 1.6062 - val_accuracy: 0.6449
Epoch 6/10
32/32 [==============================] - 6s 191ms/step - loss: 0.0081 - accuracy: 1.0000 - val_loss: 2.6666 - val_accuracy: 0.6391
Epoch 7/10
32/32 [==============================] - 6s 184ms/step - loss: 0.0035 - accuracy: 1.0000 - val_loss: 2.3257 - val_accuracy: 0.6408
Epoch

**Using an `Embedding` layer with masking enabled**

In [27]:
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = layers.Embedding(
    input_dim=max_tokens, output_dim=256, mask_zero=True)(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="relu")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="adam",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()



Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_5 (Embedding)     (None, None, 256)         2560000   
                                                                 
 bidirectional_7 (Bidirecti  (None, 64)                73984     
 onal)                                                           
                                                                 
 dropout_7 (Dropout)         (None, 64)                0         
                                                                 
 dense_7 (Dense)             (None, 1)                 65        
                                                                 
Total params: 2634049 (10.05 MB)
Trainable params: 2634049 (10.05 MB)
Non-trainable params: 0 (0.00 Byte)
___________________

In [28]:
callbacks = [
    keras.callbacks.ModelCheckpoint("embeddings_bidir_gru_with_masking.keras",
                                    save_best_only=True)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)
model = keras.models.load_model("embeddings_bidir_gru_with_masking.keras")
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Epoch 1/10
32/32 [==============================] - 18s 333ms/step - loss: 1.1140 - accuracy: 0.5110 - val_loss: 0.6888 - val_accuracy: 0.5429
Epoch 2/10
32/32 [==============================] - 7s 229ms/step - loss: 0.5693 - accuracy: 0.7440 - val_loss: 0.6514 - val_accuracy: 0.6176
Epoch 3/10
32/32 [==============================] - 7s 238ms/step - loss: 0.3120 - accuracy: 0.9220 - val_loss: 4.8207 - val_accuracy: 0.6131
Epoch 4/10
32/32 [==============================] - 7s 215ms/step - loss: 0.4042 - accuracy: 0.8530 - val_loss: 0.9986 - val_accuracy: 0.5871
Epoch 5/10
32/32 [==============================] - 6s 202ms/step - loss: 0.2045 - accuracy: 0.9090 - val_loss: 1.4468 - val_accuracy: 0.5681
Epoch 6/10
32/32 [==============================] - 7s 210ms/step - loss: 0.0428 - accuracy: 0.9930 - val_loss: 1.3126 - val_accuracy: 0.5824
Epoch 7/10
32/32 [==============================] - 6s 198ms/step - loss: 0.0190 - accuracy: 1.0000 - val_loss: 1.4464 - val_accuracy: 0.5846
Epoch

In [29]:
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = embedding_layer(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="relu")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="adam",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

callbacks = [
    keras.callbacks.ModelCheckpoint("glove_embeddings_sequence_model.keras",
                                    save_best_only=True,
                                    monitor="val_loss")
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)
model = keras.models.load_model("glove_embeddings_sequence_model.keras")
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Model: "model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 256)         2560000   
                                                                 
 bidirectional_8 (Bidirecti  (None, 64)                73984     
 onal)                                                           
                                                                 
 dropout_8 (Dropout)         (None, 64)                0         
                                                                 
 dense_8 (Dense)             (None, 1)                 65        
                                                                 
Total params: 2634049 (10.05 MB)
Trainable params: 2634049 (10.05 MB)
Non-trainable params: 0 (0.00 Byte)
___________________

# Training Sample Size: 1250

In [ ]:
shutil.rmtree('aclImdb/train_n')

In [34]:
for category in ("neg", "pos"):
    os.makedirs(train_n / category)
    files = os.listdir(train_dir / category)
    num_train_samples=1250
    train_files = files[-num_train_samples:]
    for fname in train_files:
        shutil.move(train_dir / category / fname,
                    train_n / category / fname)

train_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train_n", batch_size=batch_size
)
val_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/val", batch_size=batch_size
)
test_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/test", batch_size=batch_size
)
text_only_train_ds = train_ds.map(lambda x, y: x)

Found 2500 files belonging to 2 classes.
Found 20000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [35]:
from tensorflow.keras import layers

max_length = 150
max_tokens = 10000
text_vectorization = layers.TextVectorization(
    max_tokens=max_tokens,
    output_mode="int",
    output_sequence_length=max_length,
)
text_vectorization.adapt(text_only_train_ds)

int_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
int_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
int_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

**A sequence model built on one-hot encoded vector sequences**

In [36]:
import tensorflow as tf
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = tf.one_hot(inputs, depth=max_tokens)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="relu")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="adam",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

Model: "model_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_10 (InputLayer)       [(None, None)]            0         
                                                                 
 tf.one_hot_2 (TFOpLambda)   (None, None, 10000)       0         
                                                                 
 bidirectional_9 (Bidirecti  (None, 64)                2568448   
 onal)                                                           
                                                                 
 dropout_9 (Dropout)         (None, 64)                0         
                                                                 
 dense_9 (Dense)             (None, 1)                 65        
                                                                 
Total params: 2568513 (9.80 MB)
Trainable params: 2568513 (9.80 MB)
Non-trainable params: 0 (0.00 Byte)
_____________________

**Training a first basic sequence model**

In [37]:
callbacks = [
    keras.callbacks.ModelCheckpoint("one_hot_bidir_lstm.keras",
                                    save_best_only=True,
                                    monitor="val_loss")
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)
model = keras.models.load_model("one_hot_bidir_lstm.keras")
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Epoch 1/10
79/79 [==============================] - 14s 135ms/step - loss: 0.9722 - accuracy: 0.5256 - val_loss: 0.6776 - val_accuracy: 0.5495
Epoch 2/10
79/79 [==============================] - 10s 124ms/step - loss: 0.6703 - accuracy: 0.7000 - val_loss: 1.9686 - val_accuracy: 0.7257
Epoch 3/10
79/79 [==============================] - 10s 126ms/step - loss: 0.6336 - accuracy: 0.6464 - val_loss: 0.6307 - val_accuracy: 0.6776
Epoch 4/10
79/79 [==============================] - 10s 123ms/step - loss: 0.4990 - accuracy: 0.7716 - val_loss: 0.9107 - val_accuracy: 0.5013
Epoch 5/10
79/79 [==============================] - 10s 126ms/step - loss: 0.3542 - accuracy: 0.8608 - val_loss: 0.6135 - val_accuracy: 0.7559
Epoch 6/10
79/79 [==============================] - 10s 124ms/step - loss: 0.1375 - accuracy: 0.9604 - val_loss: 0.8224 - val_accuracy: 0.7368
Epoch 7/10
79/79 [==============================] - 10s 124ms/step - loss: 0.0490 - accuracy: 0.9884 - val_loss: 1.0777 - val_accuracy: 0.7616

**Model that uses an `Embedding` layer trained from scratch**

In [38]:
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = layers.Embedding(input_dim=max_tokens, output_dim=256)(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="relu")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="adam",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()



Model: "model_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_11 (InputLayer)       [(None, None)]            0         
                                                                 
 embedding_6 (Embedding)     (None, None, 256)         2560000   
                                                                 
 bidirectional_10 (Bidirect  (None, 64)                73984     
 ional)                                                          
                                                                 
 dropout_10 (Dropout)        (None, 64)                0         
                                                                 
 dense_10 (Dense)            (None, 1)                 65        
                                                                 
Total params: 2634049 (10.05 MB)
Trainable params: 2634049 (10.05 MB)
Non-trainable params: 0 (0.00 Byte)
__________________

In [39]:
callbacks = [
    keras.callbacks.ModelCheckpoint("embeddings_bidir_gru.keras",
                                    save_best_only=True,
                                    monitor="val_loss")
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)
model = keras.models.load_model("embeddings_bidir_gru.keras")
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Epoch 1/10
79/79 [==============================] - 15s 152ms/step - loss: 0.8148 - accuracy: 0.5344 - val_loss: 0.6687 - val_accuracy: 0.5749
Epoch 2/10
79/79 [==============================] - 10s 132ms/step - loss: 0.5007 - accuracy: 0.7944 - val_loss: 0.6370 - val_accuracy: 0.6514
Epoch 3/10
79/79 [==============================] - 8s 106ms/step - loss: 0.3206 - accuracy: 0.9032 - val_loss: 1.4512 - val_accuracy: 0.5698
Epoch 4/10
79/79 [==============================] - 7s 93ms/step - loss: 0.2178 - accuracy: 0.9144 - val_loss: 1.3862 - val_accuracy: 0.6866
Epoch 5/10
79/79 [==============================] - 7s 87ms/step - loss: 0.0671 - accuracy: 0.9820 - val_loss: 1.8814 - val_accuracy: 0.6852
Epoch 6/10
79/79 [==============================] - 6s 79ms/step - loss: 0.0360 - accuracy: 0.9900 - val_loss: 2.2309 - val_accuracy: 0.6866
Epoch 7/10
79/79 [==============================] - 6s 77ms/step - loss: 0.0193 - accuracy: 0.9948 - val_loss: 2.4372 - val_accuracy: 0.6967
Epoch 8/

**Using an `Embedding` layer with masking enabled**

In [40]:
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = layers.Embedding(
    input_dim=max_tokens, output_dim=256, mask_zero=True)(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="relu")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="adam",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()



Model: "model_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_12 (InputLayer)       [(None, None)]            0         
                                                                 
 embedding_7 (Embedding)     (None, None, 256)         2560000   
                                                                 
 bidirectional_11 (Bidirect  (None, 64)                73984     
 ional)                                                          
                                                                 
 dropout_11 (Dropout)        (None, 64)                0         
                                                                 
 dense_11 (Dense)            (None, 1)                 65        
                                                                 
Total params: 2634049 (10.05 MB)
Trainable params: 2634049 (10.05 MB)
Non-trainable params: 0 (0.00 Byte)
__________________

In [41]:
callbacks = [
    keras.callbacks.ModelCheckpoint("embeddings_bidir_gru_with_masking.keras",
                                    save_best_only=True)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)
model = keras.models.load_model("embeddings_bidir_gru_with_masking.keras")
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Epoch 1/10
79/79 [==============================] - 23s 202ms/step - loss: 0.8879 - accuracy: 0.5508 - val_loss: 0.6447 - val_accuracy: 0.6463
Epoch 2/10
79/79 [==============================] - 10s 126ms/step - loss: 0.4378 - accuracy: 0.8340 - val_loss: 0.8569 - val_accuracy: 0.7694
Epoch 3/10
79/79 [==============================] - 8s 105ms/step - loss: 0.1946 - accuracy: 0.9544 - val_loss: 1.2790 - val_accuracy: 0.7678
Epoch 4/10
79/79 [==============================] - 8s 101ms/step - loss: 0.2898 - accuracy: 0.9048 - val_loss: 1.0174 - val_accuracy: 0.7066
Epoch 5/10
79/79 [==============================] - 7s 94ms/step - loss: 0.0588 - accuracy: 0.9876 - val_loss: 1.6326 - val_accuracy: 0.7351
Epoch 6/10
79/79 [==============================] - 7s 93ms/step - loss: 0.0183 - accuracy: 0.9964 - val_loss: 1.7263 - val_accuracy: 0.7356
Epoch 7/10
79/79 [==============================] - 7s 89ms/step - loss: 0.0178 - accuracy: 0.9984 - val_loss: 1.5227 - val_accuracy: 0.7230
Epoch 8

In [42]:
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = embedding_layer(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="adam",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

callbacks = [
    keras.callbacks.ModelCheckpoint("glove_embeddings_sequence_model.keras",
                                    save_best_only=True,
                                    monitor="val_loss")
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)
model = keras.models.load_model("glove_embeddings_sequence_model.keras")
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Model: "model_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_13 (InputLayer)       [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 256)         2560000   
                                                                 
 bidirectional_12 (Bidirect  (None, 64)                73984     
 ional)                                                          
                                                                 
 dropout_12 (Dropout)        (None, 64)                0         
                                                                 
 dense_12 (Dense)            (None, 1)                 65        
                                                                 
Total params: 2634049 (10.05 MB)
Trainable params: 2634049 (10.05 MB)
Non-trainable params: 0 (0.00 Byte)
__________________

# Training Sample Size: 2500

In [ ]:
shutil.rmtree('aclImdb/train_n')

In [47]:
for category in ("neg", "pos"):
    os.makedirs(train_n / category)
    files = os.listdir(train_dir / category)
    num_train_samples=2500
    train_files = files[-num_train_samples:]
    for fname in train_files:
        shutil.move(train_dir / category / fname,
                    train_n / category / fname)

train_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train_n", batch_size=batch_size
)
val_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/val", batch_size=batch_size
)
test_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/test", batch_size=batch_size
)
text_only_train_ds = train_ds.map(lambda x, y: x)

Found 5000 files belonging to 2 classes.
Found 20000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [48]:
from tensorflow.keras import layers

max_length = 150
max_tokens = 10000
text_vectorization = layers.TextVectorization(
    max_tokens=max_tokens,
    output_mode="int",
    output_sequence_length=max_length,
)
text_vectorization.adapt(text_only_train_ds)

int_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
int_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
int_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

**A sequence model built on one-hot encoded vector sequences**

In [49]:
import tensorflow as tf
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = tf.one_hot(inputs, depth=max_tokens)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="relu")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="adam",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

Model: "model_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_14 (InputLayer)       [(None, None)]            0         
                                                                 
 tf.one_hot_3 (TFOpLambda)   (None, None, 10000)       0         
                                                                 
 bidirectional_13 (Bidirect  (None, 64)                2568448   
 ional)                                                          
                                                                 
 dropout_13 (Dropout)        (None, 64)                0         
                                                                 
 dense_13 (Dense)            (None, 1)                 65        
                                                                 
Total params: 2568513 (9.80 MB)
Trainable params: 2568513 (9.80 MB)
Non-trainable params: 0 (0.00 Byte)
____________________

**Training a first basic sequence model**

In [50]:
callbacks = [
    keras.callbacks.ModelCheckpoint("one_hot_bidir_lstm.keras",
                                    save_best_only=True,
                                    monitor="val_loss")
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)
model = keras.models.load_model("one_hot_bidir_lstm.keras")
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Epoch 1/10
157/157 [==============================] - 15s 78ms/step - loss: 0.8516 - accuracy: 0.5568 - val_loss: 0.6375 - val_accuracy: 0.7151
Epoch 2/10
157/157 [==============================] - 11s 73ms/step - loss: 0.5591 - accuracy: 0.7310 - val_loss: 1.0986 - val_accuracy: 0.5064
Epoch 3/10
157/157 [==============================] - 11s 73ms/step - loss: 0.3643 - accuracy: 0.8638 - val_loss: 0.8260 - val_accuracy: 0.8029
Epoch 4/10
157/157 [==============================] - 11s 73ms/step - loss: 0.3853 - accuracy: 0.8558 - val_loss: 0.5596 - val_accuracy: 0.7346
Epoch 5/10
157/157 [==============================] - 11s 73ms/step - loss: 0.2971 - accuracy: 0.8838 - val_loss: 0.6625 - val_accuracy: 0.7796
Epoch 6/10
157/157 [==============================] - 11s 73ms/step - loss: 0.1265 - accuracy: 0.9690 - val_loss: 1.1110 - val_accuracy: 0.8191
Epoch 7/10
157/157 [==============================] - 11s 73ms/step - loss: 0.0682 - accuracy: 0.9870 - val_loss: 1.3349 - val_accuracy:

**Model that uses an `Embedding` layer trained from scratch**

In [51]:
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = layers.Embedding(input_dim=max_tokens, output_dim=256)(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="relu")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="adam",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()



Model: "model_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_15 (InputLayer)       [(None, None)]            0         
                                                                 
 embedding_8 (Embedding)     (None, None, 256)         2560000   
                                                                 
 bidirectional_14 (Bidirect  (None, 64)                73984     
 ional)                                                          
                                                                 
 dropout_14 (Dropout)        (None, 64)                0         
                                                                 
 dense_14 (Dense)            (None, 1)                 65        
                                                                 
Total params: 2634049 (10.05 MB)
Trainable params: 2634049 (10.05 MB)
Non-trainable params: 0 (0.00 Byte)
__________________

In [52]:
callbacks = [
    keras.callbacks.ModelCheckpoint("embeddings_bidir_gru.keras",
                                    save_best_only=True,
                                    monitor="val_loss")
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)
model = keras.models.load_model("embeddings_bidir_gru.keras")
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Epoch 1/10
157/157 [==============================] - 22s 118ms/step - loss: 0.8194 - accuracy: 0.5720 - val_loss: 2.3112 - val_accuracy: 0.6988
Epoch 2/10
157/157 [==============================] - 12s 73ms/step - loss: 0.5761 - accuracy: 0.7314 - val_loss: 0.5923 - val_accuracy: 0.7008
Epoch 3/10
157/157 [==============================] - 9s 55ms/step - loss: 0.3440 - accuracy: 0.8676 - val_loss: 0.9168 - val_accuracy: 0.7783
Epoch 4/10
157/157 [==============================] - 8s 51ms/step - loss: 0.2349 - accuracy: 0.9322 - val_loss: 1.4336 - val_accuracy: 0.7509
Epoch 5/10
157/157 [==============================] - 7s 47ms/step - loss: 0.1336 - accuracy: 0.9690 - val_loss: 1.5540 - val_accuracy: 0.7707
Epoch 6/10
157/157 [==============================] - 8s 52ms/step - loss: 0.0771 - accuracy: 0.9898 - val_loss: 1.8127 - val_accuracy: 0.7754
Epoch 7/10
157/157 [==============================] - 7s 45ms/step - loss: 0.0604 - accuracy: 0.9906 - val_loss: 1.8832 - val_accuracy: 0.7

**Using an `Embedding` layer with masking enabled**

In [53]:
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = layers.Embedding(
    input_dim=max_tokens, output_dim=256, mask_zero=True)(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="relu")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="adam",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()



Model: "model_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_16 (InputLayer)       [(None, None)]            0         
                                                                 
 embedding_9 (Embedding)     (None, None, 256)         2560000   
                                                                 
 bidirectional_15 (Bidirect  (None, 64)                73984     
 ional)                                                          
                                                                 
 dropout_15 (Dropout)        (None, 64)                0         
                                                                 
 dense_15 (Dense)            (None, 1)                 65        
                                                                 
Total params: 2634049 (10.05 MB)
Trainable params: 2634049 (10.05 MB)
Non-trainable params: 0 (0.00 Byte)
__________________

In [54]:
callbacks = [
    keras.callbacks.ModelCheckpoint("embeddings_bidir_gru_with_masking.keras",
                                    save_best_only=True)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)
model = keras.models.load_model("embeddings_bidir_gru_with_masking.keras")
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Epoch 1/10
157/157 [==============================] - 28s 129ms/step - loss: 0.7560 - accuracy: 0.5732 - val_loss: 0.6146 - val_accuracy: 0.6883
Epoch 2/10
157/157 [==============================] - 13s 83ms/step - loss: 0.4621 - accuracy: 0.8130 - val_loss: 0.6295 - val_accuracy: 0.7498
Epoch 3/10
157/157 [==============================] - 10s 63ms/step - loss: 0.2510 - accuracy: 0.9340 - val_loss: 0.9425 - val_accuracy: 0.7531
Epoch 4/10
157/157 [==============================] - 9s 56ms/step - loss: 0.2841 - accuracy: 0.9124 - val_loss: 1.2972 - val_accuracy: 0.7258
Epoch 5/10
157/157 [==============================] - 8s 54ms/step - loss: 0.1085 - accuracy: 0.9766 - val_loss: 1.5621 - val_accuracy: 0.7710
Epoch 6/10
157/157 [==============================] - 8s 51ms/step - loss: 0.0545 - accuracy: 0.9948 - val_loss: 1.6743 - val_accuracy: 0.7758
Epoch 7/10
157/157 [==============================] - 8s 49ms/step - loss: 0.0452 - accuracy: 0.9964 - val_loss: 1.8152 - val_accuracy: 0.

In [55]:
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = embedding_layer(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="adam",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

callbacks = [
    keras.callbacks.ModelCheckpoint("glove_embeddings_sequence_model.keras",
                                    save_best_only=True,
                                    monitor="val_loss")
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)
model = keras.models.load_model("glove_embeddings_sequence_model.keras")
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Model: "model_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_17 (InputLayer)       [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 256)         2560000   
                                                                 
 bidirectional_16 (Bidirect  (None, 64)                73984     
 ional)                                                          
                                                                 
 dropout_16 (Dropout)        (None, 64)                0         
                                                                 
 dense_16 (Dense)            (None, 1)                 65        
                                                                 
Total params: 2634049 (10.05 MB)
Trainable params: 2634049 (10.05 MB)
Non-trainable params: 0 (0.00 Byte)
__________________